# Case Study - The Current - Part 2

* The Current is an alternative radio station
* We will pull information about the play list.

# Step 0 - Insert current progress

Copy over all the relevant code from part 1 of the lab.

http://www.thecurrent.org/playlist/2020-10-14/01

In [1]:
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace
from composable import from_toolz as tlz
import requests
from bs4 import BeautifulSoup
from composable.string import split
from composable import pipeable
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling


In [2]:
s = requests.Session() # Start a session
r = s.get('https://www.thecurrent.org/playlist/2020-10-14/01') # Get a static page
the_current_radio = BeautifulSoup(r.content, "html.parser")

# Pull off the song start time

1. Inspect the element
    1. This one is tricky
    2. Time tag does not have a tag, but
    3. The surrounding div does have a class
2. Identify the html tag and class
3. Use `find_all` to make a list of all relevant tags
4. Pull off an example case
5. Write a function that extracts the start time.
6. Write a single pipe to extract the start time.
7. Confirm you have the right number of times.
8. Package your code in a function called `get_start_time`

In [3]:
import re
time = re.compile('#song\d+')
(the_current_radio
 >> find('a', attrs = {'href':time})
 >> children
 >> tlz.get(1)
 >> get_text
 >> split(' ')
 >> tlz.get(2)
)

'1:57'

In [4]:
(the_current_radio
 >> find_all('a', attrs = {'href':time})
 >> map(children)
 >> map(tlz.get(1))
 >> map(get_text)
 >> map(split(' '))
 >> map(tlz.get(2))
)

['1:57',
 '1:53',
 '1:50',
 '1:47',
 '1:45',
 '1:41',
 '1:37',
 '1:33',
 '1:28',
 '1:24',
 '1:21',
 '1:16',
 '1:13',
 '1:09',
 '1:05',
 '1:01']

In [5]:
get_start_time = pipeable(lambda soup: soup
                          >> find_all('a', attrs = {'href':time})
                          >> map(children)
                          >> map(tlz.get(1))
                          >> map(get_text)
                          >> map(split(' '))
                          >> map(tlz.get(2))
                         )

In [18]:
the_current_radio >> get_start_time

['1:57',
 '1:53',
 '1:50',
 '1:47',
 '1:45',
 '1:41',
 '1:37',
 '1:33',
 '1:28',
 '1:24',
 '1:21',
 '1:16',
 '1:13',
 '1:09',
 '1:05',
 '1:01']

# Pull address of the album art image address

Follow a similar process to pull off the web address of the album cover image. 


In [7]:
get_src = pipeable(lambda src: src >> tlz.get('data-src') 
                               if (src >> tlz.get('src')) == '' 
                               else src >> tlz.get('src'))

In [8]:
(the_current_radio
 >> find('img', attrs = {'class':'album-art'})
 >> get_src
)

'https://albumart.publicradio.org/mb/1e/1ea97022-27ba-4bf5-8ab5-4ae76836e1c1_03df.jpg'

In [10]:
get_album_art = pipeable(lambda soup: soup
                          >> find_all('img', attrs = {'class':'album-art'})
                          >> map(get_src)
                         )

In [17]:
the_current_radio >> get_album_art

['https://albumart.publicradio.org/mb/1e/1ea97022-27ba-4bf5-8ab5-4ae76836e1c1_03df.jpg',
 'https://albumart.publicradio.org/mb/12/120c786d-a3b2-3c19-b4ff-2b7b3b4435bf_6f33.jpg',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/bc/bc3b5116-300a-3696-919d-bca466a3f143_e813.jpg',
 'https://albumart.publicradio.org/mb/fe/feeac938-9cb1-4e36-9b98-4ec2ad9a65c3_1a08.jpg',
 'https://albumart.publicradio.org/mb/16/1608212a-558a-4d20-bc01-3fc8b82e6058_6f10.jpg',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 '/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png',
 'https://albumart.publicradio.org/mb/9c/9c8f9285-

# Putting it all together

* Make a function for each of the previous steps
* Make an overall function
    * input is a soup
    * output is a list of lists

**Hint:** You should use `zip` to put all the information together.

# Final product is in lab 5_1_1